## Import some prerequirement

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# Import My Packages.
import pandas as pd
import seaborn as sn
from tensorflow import keras                                              # This is used for building model.
from tensorflow.keras.callbacks import TensorBoard                        # This is used for visualizing model training procedure.
from sklearn.metrics import confusion_matrix, classification_report

## Set hyperparameters

In [ ]:
IMG_WIDTH = 28                                                            # The width of each image in dataset.
IMG_HEIGHT = 28                                                           # The height of each image in dataset.
EPOCHS = 100                                                              # Total Number of epoch in training procedure.
BATCH_SIZE= 128                                                           # The size of each batch in SGD optimizer.
n_classes = 10                                                            # Number of classes which model should predict.

## Initialize train and test data

In [ ]:
data = np.load('mnist.npz')
[x_train, y_train, x_test, y_test] = data['x_train'], data['y_train'], data['x_test'], data['y_test']

## Show dataset

In [ ]:
plt.figure(figsize=(25,4))
for i in range(8):
    plt.subplot(1,8,i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(y_train[i])

In [ ]:
# Normalize input images.
x_train = x_train.astype(float) / 255.
x_test  = x_test.astype(float) / 255.
# Convert each final expected class to one-hot representation.
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

## Define model

In [ ]:
# In this section we build our model...
# which has only 3 Dense layers according to question.
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT)))     # Define the input shape to model.
    model.add(keras.layers.Flatten())                                # Convert input 2D images to 1D vector.
    # Three mentioned Dense layers end with the size 10.
    model.add(keras.layers.Dense(units=256, activation='relu'))
    model.add(keras.layers.Dense(units=64, activation='relu'))
    model.add(keras.layers.Dense(units=10, activation='softmax'))
    return model

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
# Run TensorBoard.
%tensorboard --logdir logs

## Compile and fit model

In [ ]:
AdaGrad_model = build_model()        # Build AdaGrad model.

# Compile AdaGrad model.
AdaGrad_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adagrad(), metrics=['accuracy'])

# Initializing ModelCheckPoint callbacks for AdaGrad model.
AdaGrad_modelcheckpoint = tf.keras.callbacks.ModelCheckpoint('best_models/AdaGrad.hdf5',
                                                             monitor="val_accuracy",
                                                             save_best_only=True)
# Initializing TensorBoard callbacks for AdaGrad model.
AdaGrad_tensorboard = TensorBoard(log_dir='logs/AdaGrad')

# Gathering two above callbacks in one callback.
AdaGrad_callbacks = [
    AdaGrad_modelcheckpoint,
    AdaGrad_tensorboard
]

# Training AdaGrad model with defined callbacks.
AdaGrad_history = AdaGrad_model.fit(x_train, y_train,
                                    validation_data=(x_test, y_test),
                                    batch_size=BATCH_SIZE,
                                    epochs=EPOCHS,
                                    shuffle=True,
                                    verbose=0,
                                    callbacks=AdaGrad_callbacks)

In [ ]:
RMSProp_model = build_model()        # Build RMSProp model.

# Compile RMSProp model.
RMSProp_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])

# Initializing ModelCheckPoint callbacks for RMSProp model.
RMSProp_modelcheckpoint = tf.keras.callbacks.ModelCheckpoint('best_models/RMSProp.hdf5',
                                                             monitor="val_accuracy",
                                                             save_best_only=True)
# Initializing TensorBoard callbacks for RMSProp model.
RMSProp_tensorboard = TensorBoard(log_dir='logs/RMSProp')

# Gathering two above callbacks in one callback.
RMSProp_callbacks = [
    RMSProp_modelcheckpoint,
    RMSProp_tensorboard
]

# Training RMSProp model with defined callbacks.
RMSProp_history = RMSProp_model.fit(x_train, y_train,
                                    validation_data=(x_test, y_test),
                                    batch_size=BATCH_SIZE,
                                    epochs=EPOCHS,
                                    shuffle=True,
                                    verbose=0,
                                    callbacks=RMSProp_callbacks)

In [ ]:
Adam_model = build_model()        # Build Adam model.

# Compile Adam model.
Adam_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Initializing ModelCheckPoint callbacks for Adam model.
Adam_modelcheckpoint = tf.keras.callbacks.ModelCheckpoint('best_models/Adam.hdf5',
                                                          monitor="val_accuracy",
                                                          save_best_only=True)
# Initializing TensorBoard callbacks for Adam model.
Adam_tensorboard = TensorBoard(log_dir='logs/Adam')

# Gathering two above callbacks in one callback.
Adam_callbacks = [
    Adam_modelcheckpoint,
    Adam_tensorboard
]

# Training Adam model with defined callbacks.
Adam_history = Adam_model.fit(x_train, y_train,
                              validation_data=(x_test, y_test),
                              batch_size=BATCH_SIZE,
                              epochs=EPOCHS,
                              shuffle=True,
                              verbose=0,
                              callbacks=Adam_callbacks)

## Load and compile best model

In [ ]:
# Load best models.
Loaded_AdaGrad_model = keras.models.load_model('best_models/AdaGrad.hdf5')
Loaded_RMSProp_model = keras.models.load_model('best_models/RMSProp.hdf5')
Loaded_Adam_model = keras.models.load_model('best_models/Adam.hdf5')
# Compile best models.
Loaded_AdaGrad_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adagrad(), metrics=['accuracy'])
Loaded_RMSProp_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])
Loaded_Adam_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

## Evaluate best model

In [ ]:
# Evaluate best models.
print('AdaGrad Evaluation Result:')
print(Loaded_AdaGrad_model.evaluate(x_test, y_test))
print('RMSProp Evaluation Result:')
print(Loaded_RMSProp_model.evaluate(x_test, y_test))
print('Adam Evaluation Result:')
print(Loaded_Adam_model.evaluate(x_test, y_test))

## Get Reports

In [ ]:
# Implementing function that report the evaluation of models with confusion matrix and classification report.
def evaluation(y_test, y_pred, title):
    cr = classification_report(y_test, y_pred)
    print(title + ' Evaluation Result - Classification Report:')
    print(cr)
    print(title + ' Evaluation Result - Confusion Matrix:')
    cm = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(cm, range(10), range(10))
    plt.figure(figsize = (10, 7))
    sn.set(font_scale = 1.4)
    sn.heatmap(df_cm , annot = True, annot_kws = {"size" : 16}, fmt = 'g') # font size
    plt.show()
    

In [ ]:
# Evaluating AdaGrad Optimization.
AdaGrad_predict = np.argmax(Loaded_AdaGrad_model.predict(x_test), axis=1)
AdaGrad_expected = np.argmax(y_test, axis=1)
evaluation(AdaGrad_expected, AdaGrad_predict, 'AdaGrad')

In [ ]:
# Evaluating RMSProp Optimization.
RMSProp_predict = np.argmax(Loaded_RMSProp_model.predict(x_test), axis=1)
RMSProp_expected = np.argmax(y_test, axis=1)
evaluation(RMSProp_expected, RMSProp_predict, 'RMSProp')

In [ ]:
# Evaluating Adam Optimization.
Adam_predict = np.argmax(Loaded_Adam_model.predict(x_test), axis=1)
Adam_expected = np.argmax(y_test, axis=1)
evaluation(Adam_expected, Adam_predict, 'Adam')